In [ ]:
import socket
table = {}
parts = {}

def get(key) :
    return table.get(key)

def put(key, value) :
    table[key] = value
    
def delete(key) :
    table.pop(key, None)
    
def clear() :
    table.clear()
    
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serversocket.bind(('', 8000))
serversocket.listen(1)
while True:
    print ("Listening for connections...")
    (client, addr) = serversocket.accept()
    client.sendto(b"Connection established.\r\n", addr)
    print ("Connection established.")
    
    while True:
        message = client.recv(2048)
        if message.startswith(b"GET") or message.startswith(b"get") :           # Checks if command is GET, inclusive of GETTER
            if message.find(b" ") > 3:                    # Checks if command is GETTER
                client.sendto(b"HTTP/1.1 220 UNSUPPORTED\n\n\r", addr)
                print("HTTP/1.1 220 UNSUPPORTED")
            elif message.rfind(b" ") > 3 or message.rfind(b" ") == len(message.decode()) - 1 or message.find(b" ") == -1:
                client.sendto(b"HTTP/1.1 400 BAD_REQUEST\n\n\r", addr)        # Checks if command has incorrect parameters
                print("HTTP/1.1 400 BAD_REQUEST")
            else:
                try:                                # Attempts to find and get the key-value pair, returning None if non existent
                    print (get(message[4:].decode()))
                    client.sendto(("HTTP/1.1 200 OK\n\r" + get(message[4:].decode()) + "\r\n\n").encode(), addr)
                    print ("HTTP/1.1 200 OK")
                except:                                # Returning None will launch except block
                    print("HTTP/1.1 404. NOT FOUND")
                    client.sendto(b"HTTP/1.1 404 NOT FOUND\n\n\r", addr)

        elif message.startswith(b"PUT") or message.startswith(b"put") :                 # Checks if command is PUT
            index = message.rfind(b" ")
            if message.find(b" ") > 3:
                client.sendto(b"HTTP/1.1 220 UNSUPPORTED\n\n\r", addr)
                print("HTTP/1.1 220 UNSUPPORTED")
            elif message.rfind(b" ") == 3 or message.find(b" ", 4) < index or message.rfind(b" ") == len(message.decode()) - 1 or message.find(b" ") == -1: 
                client.sendto(b"HTTP/1.1 400 BAD_REQUEST\n\n\r", addr)      # Checks if only 1 parameter is given for PUT command
                print("HTTP/1.1 400 BAD_REQUEST")                           # Also checks if command has incorrect parameters
            else: 
                print (message[4:index].decode(), message[index + 1:].decode())
                put(message[4:index].decode(), message [index + 1:].decode())
                client.sendto(b"HTTP/1.1 200 OK\n\n\r", addr)
                print("HTTP/1.1 200 OK")

        elif message.startswith(b"DELETE") or message.startswith(b"delete"):           # Checks if command is DELETE
            if message.find(b" ") > 6 :            # Checks if command is DELETER
                print("HTTP/1.1 220 UNSUPPORTED")
                client.sendto(b"HTTP/1.1 220 UNSUPPORTED\n\n\r", addr)  
            elif message.find(b" ") == -1 or message.rfind(b" ") > 6 or message.rfind(b" ") == len(message.decode()) - 1 :  
                print("HTTP/1.1 400 BAD_REQUEST")           # Checks if command has incorrect parameters
                client.sendto(b"HTTP/1.1 400 BAD_REQUEST\n\n\r", addr)
            else:
                delete(message[7:].decode())
                client.sendto(b"HTTP/1.1 200 OK\n\n\r", addr)
                print("HTTP/1.1 200 OK")            

        elif message.startswith(b"CLEAR") or message.startswith(b"clear"):             # Checks if command is CLEAR
            if len(message.decode()) != 5 :                 # Checks if command is CLEAR and only CLEAR
                if message[5] == 32 :            #Checks if status should be UNSUPPORTED or BAD_REQUEST
                    print("HTTP/1.1 400 BAD_REQUEST")  
                    client.sendto(b"HTTP/1.1 400 BAD_REQUEST\n\n\r", addr)
                else :
                    print(message[5])
                    print("HTTP/1.1 220 UNSUPPORTED")
                    client.sendto(b"HTTP/1.1 220 UNSUPPORTED\n\n\r", addr)
            else:
                clear()
                print("HTTP/1.1 200 OK")
                client.sendto(b"HTTP/1.1 200 OK\n\n\r", addr)
        
        elif message.startswith(b"QUIT") or message.startswith(b"quit") :        # Checks if command is QUIT
            if len(message.decode()) != 4 :                # Checks if command is QUIT and only QUIT
                if message[4] == 32 :           # Checks if status should be UNSUPPORTED OR BAD_REQUEST
                    print("HTTP/1.1 400 BAD_REQUEST")
                    client.sendto(b"HTTP/1.1 400 BAD_REQUEST\n\n\r", addr)
                else :
                    print("HTTP/1.1 220 UNSUPPORTED")
                    client.sendto(b"HTTP/1.1 220 UNSUPPORTED\n\n\r", addr)
            else:
                print("HTTP/1.1 200 OK")
                print("Connection closed.")
                client.sendto(b"HTTP/1.1 200 OK\n\rConnection closed.\n\n\r", addr)
                client.shutdown(socket.SHUT_RDWR)
                client.close()
                break     
                
        else : # UNRECOGNIZED COMMANDS
            if message[0] == 255 or message[0] == 13:
                continue
            else:
                print(message)
                print("HTTP/1.1 220 UNSUPPORTED")
                client.sendto(b"HTTP/1.1 220 UNSUPPORTED\n\n\r", addr)
        